In [1]:
!pip -q install groq pypdf

In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"
os.environ["TOMTOM_API_KEY"] = "lsMxbkPAAw9Zoya1rFoe9WEZGUBJnAee"


In [3]:
import requests, os

key = os.getenv("TOMTOM_API_KEY")
url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
params = {
    "key": key,
    "point": "41.0369,28.9850"  # Taksim civarı
}

r = requests.get(url, params=params, timeout=20)
print("Status:", r.status_code)
print(r.text[:300])


Status: 200
{"flowSegmentData":{"frc":"FRC2","currentSpeed":30,"freeFlowSpeed":33,"currentTravelTime":1308,"freeFlowTravelTime":1189,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.003728529171610,"longitude":28.955802137254693},{"latitude":41.003780841406027,"longitude":28.955649


In [5]:
import google.generativeai as genai
from google.colab import userdata

# 1. API Anahtarını Tanımlama
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

# 2. Modeli Başlatma (Başına models/ eklemek daha garantidir)
model = genai.GenerativeModel("models/gemini-2.5-flash")

# 3. İçerik Oluşturma
try:
    response = model.generate_content("Esenyurt’ta yoğun trafik varsa sürüşte hangi riskler artar?")
    print(response.text)
except Exception as e:
    print(f"Bir hata oluştu: {e}")

Esenyurt gibi yoğun nüfuslu ve trafik akışı yüksek bölgelerde yaşanan trafik sıkışıklığı, sürüş sırasında birçok riski beraberinde getirir. Bu riskler hem sürücüler hem de trafikteki diğer unsurlar (yayalar, bisikletliler, acil durum araçları) için geçerlidir:

1.  **Kaza Riski Artışı:**
    *   **Arkadan Çarpma (Rear-end collisions):** Yoğun trafikte araçlar arasındaki mesafe kısalır ve ani frenlemeler sık görülür. Sürücülerin dikkatsizliği veya takip mesafesine uymaması bu tür kazaların en yaygın nedenidir.
    *   **Şerit Değiştirme ve Yan Çarpma Kazaları:** Sabırsız sürücüler, boşluk bulmak için sürekli şerit değiştirmeye çalışabilir. Dar alanlarda ve düşük hızlarda yapılan bu manevralar, yan çarpışmalara yol açabilir.
    *   **Düşük Hızlı Çarpışmalar:** Hız düşük olsa bile, araçların tampon tampona veya hafif sürtünmelerle çarpışması maddi hasara ve zaman kaybına neden olur.

2.  **Sürücü Stresi ve Yorgunluğu:**
    *   **Yol Öfkesi ve Agresif Sürüş:** Uzun süre trafikte kalmak, 

In [6]:
import google.generativeai as genai
# API anahtarınızı yapılandırdığınızdan emin olun

for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-prev

In [7]:
# ============================================================
# AKILLI ULAŞIM & TRAFİK OPTİMİZASYONU
# ReAct + Multi-RAG + API Tools ✅
# Benchmark-ODAKLI STABLE: Forced tools + Math arg fix + Anti-hallucination
# + A/B KARŞILAŞTIRMA: (A) Senin deterministik finalin  vs  (B) Gemini final
# ============================================================

import os
import re
import json
import math
import random
import requests
import ast
import time
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

# -------------------------
# Optional Groq (demo loop için)
# -------------------------
USE_GROQ = True
try:
    from groq import Groq
except Exception:
    USE_GROQ = False

# -------------------------
# Optional Gemini (benchmark final için)
# -------------------------
USE_GEMINI = True
try:
    import google.generativeai as genai
except Exception:
    USE_GEMINI = False

# -------------------------
# PDF
# -------------------------
try:
    from pypdf import PdfReader
except Exception as e:
    raise RuntimeError("pypdf yüklü değil. 'pip install pypdf' çalıştırın.") from e


# ============================================================
# 0) Helpers
# ============================================================

def tr_normalize(text: str) -> str:
    if not text:
        return ""
    return text.replace("İ", "i").replace("I", "ı").lower().strip()

def clamp(x, a, b):
    return max(a, min(b, x))

_WORD_RE = re.compile(r"[a-zA-ZçğıöşüÇĞİÖŞÜ0-9]+", re.UNICODE)

def tokenize(text: str) -> List[str]:
    return _WORD_RE.findall(tr_normalize(text))

def safe_json(obj: Any, lim: int = 6000) -> str:
    s = json.dumps(obj, ensure_ascii=False)
    return s[:lim]


# ============================================================
# 0.1) Numeric evidence helpers (Hallucination guard)
# ============================================================

_NUM_RE = re.compile(r"\d+(?:[.,]\d+)?")

def _canon_num_str(s: str) -> str:
    s = (s or "").strip().replace(",", ".")
    if not s:
        return s
    if "." in s:
        a, b = s.split(".", 1)
        a = a.lstrip("0") or "0"
        b = b.rstrip("0")
        if b == "":
            return a
        return f"{a}.{b}"
    return (s.lstrip("0") or "0")

def extract_numbers(text: str) -> List[str]:
    return [_canon_num_str(x) for x in _NUM_RE.findall(text or "")]

def numbers_set(texts: List[str]) -> set:
    out = set()
    for t in texts:
        for n in extract_numbers(t):
            out.add(n)
    return out


# ============================================================
# 1) PDF read + chunk
# ============================================================

def read_pdf_text(pdf_path: str, max_pages: Optional[int] = None) -> str:
    reader = PdfReader(pdf_path)
    texts = []
    n = len(reader.pages) if max_pages is None else min(len(reader.pages), max_pages)
    for i in range(n):
        texts.append(reader.pages[i].extract_text() or "")
    return "\n".join(texts)

def chunk_text(text: str, chunk_size: int = 900, overlap: int = 150) -> List[str]:
    text = re.sub(r"\n{3,}", "\n\n", text)
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(L, start + chunk_size)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = max(0, end - overlap)
        if end == L:
            break
    return chunks


# ============================================================
# 2) Simple TF-IDF store
# ============================================================

class SimpleVectorStore:
    def __init__(self, name: str):
        self.name = name
        self.docs: List[Dict[str, Any]] = []
        self.df: Dict[str, int] = {}
        self.N = 0

    def add_documents(self, chunks: List[str], metadata: Dict[str, Any]):
        for idx, ch in enumerate(chunks):
            tokens = tokenize(ch)
            tf: Dict[str, int] = {}
            for w in tokens:
                tf[w] = tf.get(w, 0) + 1

            for w in set(tf.keys()):
                self.df[w] = self.df.get(w, 0) + 1

            self.docs.append({
                "text": ch,
                "tf": tf,
                "meta": {**metadata, "chunk_id": idx}
            })
            self.N += 1

    def _tfidf(self, tf: Dict[str, int]) -> Dict[str, float]:
        vec: Dict[str, float] = {}
        for w, c in tf.items():
            idf = math.log((self.N + 1) / (self.df.get(w, 0) + 1)) + 1.0
            vec[w] = (1.0 + math.log(1 + c)) * idf
        return vec

    def _cosine(self, a: Dict[str, float], b: Dict[str, float]) -> float:
        inter = set(a.keys()) & set(b.keys())
        num = sum(a[w] * b[w] for w in inter)
        da = math.sqrt(sum(v * v for v in a.values()))
        db = math.sqrt(sum(v * v for v in b.values()))
        if da == 0 or db == 0:
            return 0.0
        return num / (da * db)

    def search(self, query: str, top_k: int = 4) -> List[Dict[str, Any]]:
        q_tokens = tokenize(query)
        q_tf: Dict[str, int] = {}
        for w in q_tokens:
            q_tf[w] = q_tf.get(w, 0) + 1
        q_vec = self._tfidf(q_tf)

        scored: List[Tuple[float, Dict[str, Any]]] = []
        for d in self.docs:
            d_vec = self._tfidf(d["tf"])
            s = self._cosine(q_vec, d_vec)
            scored.append((s, d))

        scored.sort(key=lambda x: x[0], reverse=True)
        return [{
            "score": float(s),
            "text": d["text"],
            "meta": d["meta"]
        } for s, d in scored[:top_k]]


# ============================================================
# 3) MultiRAG
# ============================================================

class MultiRAG:
    def __init__(self):
        self.stores: Dict[str, SimpleVectorStore] = {}

    def ingest_pdf(self, pdf_path: str, doc_type: str, chunk_size: int = 900, overlap: int = 150):
        text = read_pdf_text(pdf_path)
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
        if doc_type not in self.stores:
            self.stores[doc_type] = SimpleVectorStore(doc_type)

        self.stores[doc_type].add_documents(
            chunks, metadata={"source": os.path.basename(pdf_path), "doc_type": doc_type}
        )

    def tool_search(self, query: str, doc_type: str, top_k: int = 4) -> Dict[str, Any]:
        if doc_type not in self.stores:
            return {"ok": False, "error": f"RAG store yok: {doc_type}", "results": []}

        results = self.stores[doc_type].search(query, top_k=top_k)
        best = results[0]["score"] if results else 0.0
        sufficient = best >= 0.08
        return {"ok": True, "doc_type": doc_type, "sufficient": sufficient, "best_score": best, "results": results}


# ============================================================
# 4) API Tools
# ============================================================

def live_traffic_api_tomtom(area_query: str) -> Dict[str, Any]:
    api_key = os.getenv("TOMTOM_API_KEY", "").strip()
    if not api_key:
        return {
            "ok": True,
            "provider": "MOCK",
            "area": area_query,
            "summary": "API anahtarı bulunamadı; MOCK trafik durumu kullanıldı.",
            "traffic": {
                "congestion_pct": random.choice([0, 10, 25, 40, 60]),
                "status": random.choice(["Akıcı", "Orta", "Yoğun", "Çok Yoğun"])
            }
        }

    try:
        geo_url = "https://api.tomtom.com/search/2/geocode/{}.json".format(requests.utils.quote(area_query))
        time.sleep(0.35)
        geo_r = requests.get(geo_url, params={"key": api_key, "limit": 1, "countrySet": "TR", "language": "tr-TR"}, timeout=20)
        geo_r.raise_for_status()
        geo = geo_r.json()
        if not geo.get("results"):
            return {"ok": False, "provider": "TomTom", "error": "Konum bulunamadı."}

        pos = geo["results"][0]["position"]
        lat, lon = pos["lat"], pos["lon"]

        flow_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
        time.sleep(0.35)
        flow_r = requests.get(flow_url, params={"key": api_key, "point": f"{lat},{lon}"}, timeout=20)
        flow_r.raise_for_status()
        flow = flow_r.json()

        fsd = flow.get("flowSegmentData", {})
        cur = fsd.get("currentSpeed", None)
        free = fsd.get("freeFlowSpeed", None)

        congestion_pct = None
        if cur is not None and free not in (None, 0):
            congestion_pct = int(round((1 - (cur / free)) * 100))
            congestion_pct = clamp(congestion_pct, 0, 100)

        status = "Akıcı"
        if congestion_pct is not None:
            if congestion_pct >= 60:
                status = "Çok Yoğun"
            elif congestion_pct >= 35:
                status = "Yoğun"
            elif congestion_pct >= 15:
                status = "Orta"

        return {
            "ok": True,
            "provider": "TomTom",
            "area": area_query,
            "traffic": {
                "currentSpeed": cur,
                "freeFlowSpeed": free,
                "congestion_pct": congestion_pct,
                "status": status
            }
        }
    except Exception as e:
        return {"ok": False, "provider": "TomTom", "error": str(e)}

def route_eta_api_tomtom(start: str, end: str) -> Dict[str, Any]:
    api_key = os.getenv("TOMTOM_API_KEY", "").strip()
    if not api_key:
        return {"ok": False, "error": "TOMTOM_API_KEY bulunamadı."}

    try:
        def geocode(q: str) -> Optional[str]:
            url = f"https://api.tomtom.com/search/2/geocode/{requests.utils.quote(q)}.json"
            time.sleep(0.35)
            r = requests.get(url, params={
                "key": api_key, "limit": 1, "countrySet": "TR", "language": "tr-TR"
            }, timeout=15)
            r.raise_for_status()
            res = r.json()
            if not res.get("results"):
                return None
            p = res["results"][0]["position"]
            return f"{p['lat']},{p['lon']}"

        sp = geocode(start)
        ep = geocode(end)
        if not sp or not ep:
            return {"ok": False, "error": "Başlangıç veya varış konumu bulunamadı."}

        route_url = f"https://api.tomtom.com/routing/1/calculateRoute/{sp}:{ep}/json"
        time.sleep(0.35)
        r = requests.get(route_url, params={
            "key": api_key,
            "traffic": "true",
            "maxAlternatives": 2,
            "routeType": "fastest",
            "travelMode": "car"
        }, timeout=20)
        r.raise_for_status()
        data = r.json()

        routes = []
        for idx, rt in enumerate(data.get("routes", [])):
            summ = rt["summary"]
            routes.append({
                "alternative": idx,
                "length_km": round(summ["lengthInMeters"] / 1000, 2),
                "travel_time_min": round(summ["travelTimeInSeconds"] / 60, 1),
                "traffic_delay_min": round(summ.get("trafficDelayInSeconds", 0) / 60, 1)
            })

        return {"ok": True, "provider": "TomTom", "start": start, "end": end, "routes": routes}
    except Exception as e:
        return {"ok": False, "error": str(e)}

def math_tool(expression: str) -> Dict[str, Any]:
    expr = (expression or "").strip()
    if not expr:
        return {"ok": False, "error": "Boş ifade."}
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\)\s%<>!=&\|]+", expr):
        return {"ok": False, "error": "İfadede izin verilmeyen karakter var."}
    try:
        expr = re.sub(r"(\d+(?:\.\d+)?)\s*%", r"(\1/100)", expr)
        val = eval(expr, {"__builtins__": {}}, {})
        return {"ok": True, "value": val, "expression": expr}
    except Exception as e:
        return {"ok": False, "error": str(e)}


# ============================================================
# 5) ReAct Agent (Benchmark stable)
# ============================================================

@dataclass
class TraceStep:
    turn: int
    thought: str
    action: str
    observation: str


class SmartTrafficReActAgent:
    def __init__(self, rag: MultiRAG):
        self.rag = rag

        # ---- Groq (demo loop) ----
        self.groq_client = None
        self.groq_model = os.getenv("GROQ_MODEL", "llama-3.3-70b-versatile")
        if USE_GROQ:
            api_key = os.getenv("GROQ_API_KEY", "").strip()
            if api_key:
                self.groq_client = Groq(api_key=api_key)

        # ---- Gemini (benchmark final) ----
        self.gemini_model_name = os.getenv("GEMINI_MODEL", "gemini-1.5-pro")
        self.gemini_client_ok = False
        if USE_GEMINI:
            gkey = os.getenv("GEMINI_API_KEY", "").strip()
            if gkey:
                try:
                    genai.configure(api_key=gkey)
                    self.gemini_client_ok = True
                except Exception:
                    self.gemini_client_ok = False

        self.tools = {
            "rag_search": self._tool_rag_search,
            "live_traffic_api": self._tool_live_traffic,
            "route_eta_api": self._tool_route_eta,
            "math": self._tool_math,
        }

    # ---------- Tool wrappers ----------
    def _tool_rag_search(self, args: Dict[str, Any]) -> Dict[str, Any]:
        return self.rag.tool_search(
            query=args.get("query", ""),
            doc_type=args.get("doc_type", ""),
            top_k=int(args.get("top_k", 4))
        )

    def _tool_live_traffic(self, args: Dict[str, Any]) -> Dict[str, Any]:
        return live_traffic_api_tomtom(args.get("area", ""))

    def _tool_route_eta(self, args: Dict[str, Any]) -> Dict[str, Any]:
        return route_eta_api_tomtom(start=args.get("start", ""), end=args.get("end", ""))

    def _tool_math(self, args: Dict[str, Any]) -> Dict[str, Any]:
        expr = ""
        if isinstance(args, str):
            expr = args
        elif isinstance(args, dict):
            expr = (args.get("expression") or args.get("expr") or args.get("formula") or args.get("calc") or "").strip()
            if not expr and "operation" in args and "formula" in args:
                expr = str(args.get("formula", "")).strip()
        return math_tool(expr)

    # ---------- System prompt (only used if Groq active) ----------
    def _system_prompt(self) -> str:
        return """
Sen ReAct ajanısın. THOUGHT->ACTION->OBSERVATION döngüsüyle ilerle ve FINAL ANSWER üret.

KURALLAR:
- Yalnız şu araç isimleri: rag_search, live_traffic_api, route_eta_api, math
- ACTION argümanları MUTLAKA geçerli JSON olmalı ve stringler çift tırnaklı olmalı.
- FINAL ANSWER içindeki sayılar, OBSERVATION içinde geçmek zorunda (uydurma sayı yok).
- Çince/başka dil karıştırma; sadece Türkçe yaz.
Format:
THOUGHT: ...
ACTION: tool_name {...}
VEYA
FINAL ANSWER: ...
""".strip()

    def _llm(self, prompt: str) -> str:
        if self.groq_client is None:
            return "FINAL ANSWER: Groq yok (deterministic fallback kullanıldı)."
        resp = self.groq_client.chat.completions.create(
            model=self.groq_model,
            messages=[
                {"role": "system", "content": self._system_prompt()},
                {"role": "user", "content": prompt},
            ],
            temperature=0.2,
        )
        return resp.choices[0].message.content

    # ---------- Gemini final helper (Benchmark) ----------
    def _gemini_final_benchmark(self,
                               question: str,
                               traffic_res: Optional[Dict[str, Any]],
                               route_res: Optional[Dict[str, Any]],
                               rag_res: Optional[Dict[str, Any]],
                               math_pack: Optional[Dict[str, Any]],
                               contexts_used: List[Dict[str, Any]]) -> str:
        # Gemini yoksa fallback
        if not self.gemini_client_ok:
            return "Gemini aktif değil (GEMINI_API_KEY yok veya google-generativeai yüklü değil)."

        # RAG snippet’leri (ilk 3)
        snippets = []
        if rag_res and rag_res.get("ok"):
            for r in (rag_res.get("results") or [])[:3]:
                snippets.append({
                    "score": r.get("score"),
                    "source": (r.get("meta") or {}).get("source"),
                    "chunk_id": (r.get("meta") or {}).get("chunk_id"),
                    "text": (r.get("text") or "")[:1200]
                })

        payload = {
            "question": question,
            "traffic_observation": traffic_res,
            "route_observation": route_res,
            "rag_observation": {
                "doc_type": (rag_res or {}).get("doc_type"),
                "best_score": (rag_res or {}).get("best_score"),
                "snippets": snippets
            },
            "math_results": (math_pack or {}).get("computed", {}) if isinstance(math_pack, dict) else {},
            "rules": [
                "Sadece yukarıdaki observation/snippet içindeki sayıları kullan. Uydurma sayı yok.",
                "Soru şartları (ör: yoğunluk %25 üstü ise) sağlanmıyorsa açıkça 'şart tetiklenmedi' de.",
                "İstenen madde sayısına uy (2/3/4/5).",
                "Çıktı Türkçe olsun."
            ]
        }

        prompt = (
            "Aşağıdaki JSON gözlemlerine dayanarak tek bir 'FINAL ANSWER' üret.\n"
            "Sayı uydurma yok; sadece JSON/snippet içindeki sayıları kullan.\n"
            "Şartlar tetiklenmediyse bunu açıkça belirt.\n\n"
            f"{json.dumps(payload, ensure_ascii=False)}"
        )

        try:
            model = genai.GenerativeModel(self.gemini_model_name)
            resp = model.generate_content(prompt)
            txt = (resp.text or "").strip()
            # Bazı durumlarda Gemini "FINAL ANSWER:" yazabilir; normalize edelim
            txt = re.sub(r"^\s*FINAL\s*ANSWER\s*:\s*", "", txt, flags=re.IGNORECASE)
            return txt.strip()
        except Exception as e:
            return f"Gemini hata: {str(e)}"

    # ---------- Place cleaning ----------
    def _clean_place(self, s: str) -> str:
        raw = (s or "").strip()
        low = tr_normalize(raw)

        low = re.sub(r"^(önce|lütfen)\s+", "", low, flags=re.IGNORECASE)
        low = re.sub(r"^(tomtom ile|tomtomla|tomtom)\s+", "", low, flags=re.IGNORECASE)
        low = re.sub(r"\b(trafik etkili|rota|eta|bilgisini|bilgisi|çek|al|hesapla)\b", "", low, flags=re.IGNORECASE)
        low = re.sub(r"\s+", " ", low).strip()

        cleaned = low.title() if low else raw.strip()

        ist_hot = {"Taksim", "Kadıköy", "Esenyurt", "Beşiktaş", "Üsküdar", "Ataşehir", "Bakırköy", "Şişli", "Beylikdüzü"}
        if cleaned in ist_hot and "İstanbul" not in cleaned:
            cleaned = f"{cleaned}, İstanbul"
        return cleaned

    def _extract_route_points(self, question: str) -> Tuple[str, str]:
        q = (question or "").strip()
        m = re.search(r"(.+?)(?:'den|'dan|den|dan)\s+(.+?)(?:'e|'a|e|a)\b", q, flags=re.IGNORECASE)
        if m:
            return self._clean_place(m.group(1)), self._clean_place(m.group(2))
        m2 = re.search(r"(.+?)\s+(?:ile)\s+(.+?)\s+(?:arası|arasi)\b", q, flags=re.IGNORECASE)
        if m2:
            return self._clean_place(m2.group(1)), self._clean_place(m2.group(2))
        return "Taksim, İstanbul", "Kadıköy, İstanbul"

    # ---------- Evidence guard ----------
    def _final_has_unsupported_numbers(self, final_text: str, evidence_texts: List[str]) -> Tuple[bool, List[str]]:
        final_nums = set(extract_numbers(final_text))
        if not final_nums:
            return False, []
        evid_nums = numbers_set(evidence_texts)
        expanded = set(evid_nums)
        for n in list(evid_nums):
            if "." in n:
                expanded.add(n.split(".", 1)[0])
        unsupported = sorted([n for n in final_nums if n not in expanded])
        return (len(unsupported) > 0), unsupported

    # ---------- Deterministic helpers ----------
    def _pick_top_safety_bullets(self, rag_result: Dict[str, Any], k: int = 4) -> List[str]:
        bullets = []
        for r in (rag_result.get("results") or [])[:3]:
            txt = (r.get("text") or "")
            for line in txt.splitlines():
                line = line.strip()
                if not line:
                    continue
                if line.startswith("•") or line.startswith("-"):
                    clean = re.sub(r"^[•\-]\s*", "", line).strip()
                    if len(clean) >= 8:
                        bullets.append(clean)
                if len(bullets) >= k:
                    return bullets[:k]
        if not bullets:
            for r in (rag_result.get("results") or [])[:2]:
                txt = re.sub(r"\s+", " ", (r.get("text") or "")).strip()
                parts = re.split(r"(?<=[\.\!\?])\s+", txt)
                for p in parts:
                    p = p.strip()
                    if 15 <= len(p) <= 140:
                        bullets.append(p)
                    if len(bullets) >= k:
                        return bullets[:k]
        return bullets[:k]

    def _routes_sorted(self, route_res: Dict[str, Any]) -> List[Dict[str, Any]]:
        routes = route_res.get("routes") or []
        return sorted(routes, key=lambda r: r.get("travel_time_min", 1e9))

    def _best_route(self, route_res: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        if not route_res.get("ok"):
            return None
        rs = self._routes_sorted(route_res)
        return rs[0] if rs else None

    def _parse_speed_from_question(self, q_norm: str, default_speed: float = 90.0) -> float:
        m = re.search(r"(\d+(?:[.,]\d+)?)\s*km", q_norm)
        if m:
            return float(m.group(1).replace(",", "."))
        m2 = re.search(r"hız\s*=\s*(\d+(?:[.,]\d+)?)", q_norm)
        if m2:
            return float(m2.group(1).replace(",", "."))
        return float(default_speed)

    def _call_math(self, expr: str, tools_used: List[str], trace: List[TraceStep], evidence_texts: List[str], turn: int, why: str) -> Dict[str, Any]:
        args = {"expression": expr}
        tools_used.append("math")
        res = self.tools["math"](args)
        obs = safe_json(res)
        evidence_texts.append(obs)
        trace.append(TraceStep(turn, why, f"math {json.dumps(args, ensure_ascii=False)}", obs[:800] + ("..." if len(obs) > 800 else "")))
        return res

    def _benchmark_math_pack(self, question: str, traffic_res: Dict[str, Any], route_res: Dict[str, Any],
                             tools_used: List[str], trace: List[TraceStep], evidence_texts: List[str], start_turn: int) -> Dict[str, Any]:
        qn = tr_normalize(question)
        out: Dict[str, Any] = {"computed": {}}
        t = (traffic_res.get("traffic") or {}) if traffic_res else {}
        cong = t.get("congestion_pct")

        best = self._best_route(route_res or {}) if route_res else None
        routes_sorted = self._routes_sorted(route_res or {}) if route_res else []

        turn = start_turn

        # (A) Standart takip mesafesi: hız/2
        if any(k in qn for k in ["takip mesafes", "hızın yarısı", "yarısı kadar metre", "hizın yarısı", "yarisi kadar metre"]):
            sp = self._parse_speed_from_question(qn, default_speed=90.0)
            res = self._call_math(f"{sp}/2", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (takip mesafesi hız/2)")
            turn += 1
            if res.get("ok"):
                out["computed"]["follow_distance_m"] = float(res["value"])

        # (B) Tamponlu takip mesafesi: (hız/2)*1.3 veya +10
        if "tampon" in qn and "1.3" in qn:
            sp = self._parse_speed_from_question(qn, default_speed=60.0)
            res = self._call_math(f"({sp}/2)*1.3", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (tamponlu takip mesafesi *1.3)")
            turn += 1
            if res.get("ok"):
                out["computed"]["follow_distance_buffered_m"] = float(res["value"])

        if "+10" in qn and "takip mesafes" in qn and cong is not None:
            sp = self._parse_speed_from_question(qn, default_speed=70.0)
            res1 = self._call_math(f"{sp}/2", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (base hız/2)")
            turn += 1
            res2 = self._call_math(f"({sp}/2)+10", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (base+10)")
            turn += 1
            if res1.get("ok"):
                out["computed"]["follow_distance_base_m"] = float(res1["value"])
            if res2.get("ok"):
                out["computed"]["follow_distance_plus10_m"] = float(res2["value"])

        # (C) % hesapları: gecikme yüzdesi, gecikme/mesafe oranı vb.
        if ("oran" in qn or "yüzde" in qn or "percent" in qn or "%" in qn) and best:
            delay = best.get("traffic_delay_min")
            tt = best.get("travel_time_min")
            km = best.get("length_km")
            if delay is not None and tt not in (None, 0) and ("%" in qn or "yüzde" in qn):
                res = self._call_math(f"({delay}/{tt})*100", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (gecikme yüzdesi)")
                turn += 1
                if res.get("ok"):
                    out["computed"]["delay_pct"] = float(res["value"])
            if delay is not None and km not in (None, 0) and ("dk / km" in qn or "dk/km" in qn or "gecikme/mesafe" in qn):
                res = self._call_math(f"{delay}/{km}", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (gecikme dk/km)")
                turn += 1
                if res.get("ok"):
                    out["computed"]["delay_per_km"] = float(res["value"])

        # (D) Skor / puan / risk / stres
        if "skor" in qn and "gecikme" in qn and best:
            delay = best.get("traffic_delay_min") or 0
            res = self._call_math(f"{delay}*2", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (skor=gecikme*2)")
            turn += 1
            if res.get("ok"):
                out["computed"]["score"] = float(res["value"])

        if "stres" in qn and best:
            delay = best.get("traffic_delay_min") or 0
            cong_val = cong if cong is not None else 0
            res = self._call_math(f"{cong_val}+({delay}*5)", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (stres=yoğunluk+gecikme*5)")
            turn += 1
            if res.get("ok"):
                out["computed"]["stress"] = float(res["value"])

        if "risk" in qn and "risk =" in qn and best:
            delay = best.get("traffic_delay_min") or 0
            tt = best.get("travel_time_min") or 0
            res = self._call_math(f"({delay}*10)+({tt}/10)", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (risk=gecikme*10+süre/10)")
            turn += 1
            if res.get("ok"):
                out["computed"]["risk"] = float(res["value"])

        # (E) Rota seçim puanı: puan = süre + k*gecikme
        if "puan" in qn and "rota" in qn and routes_sorted:
            k = None
            mk = re.search(r"puan\s*=\s*süre\s*\+\s*(\d+)\s*\*\s*gecikme", qn)
            if mk:
                k = float(mk.group(1))
            if k is None:
                mk2 = re.search(r"süre\s*\+\s*(\d+)\s*\*\s*gecikme", qn)
                if mk2:
                    k = float(mk2.group(1))
            if k is None:
                k = 2.0

            per_route_scores = []
            for r in routes_sorted:
                tt = r.get("travel_time_min") or 0
                d = r.get("traffic_delay_min") or 0
                res = self._call_math(f"{tt}+({k}*{d})", tools_used, trace, evidence_texts, turn, f"Benchmark forced: math (rota puanı alt={r.get('alternative')})")
                turn += 1
                if res.get("ok"):
                    per_route_scores.append({"alternative": r.get("alternative"), "score": float(res["value"])})
            out["computed"]["route_scores"] = per_route_scores

        # (F) Risk etiketi mantığı: (yoğunluk>=35 veya gecikme>8)
        if "risk etiketi" in qn and best:
            delay = best.get("traffic_delay_min")
            cong_val = cong if cong is not None else 0
            delay_val = delay if delay is not None else 0
            res = self._call_math(f"(({cong_val}>=35) or ({delay_val}>8))", tools_used, trace, evidence_texts, turn, "Benchmark forced: math (risk etiketi koşulu)")
            turn += 1
            if res.get("ok"):
                out["computed"]["risk_flag"] = bool(res["value"])

        out["turn_end"] = turn
        return out

    def _benchmark_rag_query(self, question: str, q_norm: str, doc_type: str) -> str:
        base = "güvenli sürüş önlemleri"
        if doc_type == "SIGNS":
            base = "trafik işaretleri uyarı levhaları hız sınırı"
        elif doc_type == "REGULATION":
            base = "trafik yönetmeliği takip mesafesi hız kuralları"
        else:
            if "yağmur" in q_norm or "ıslak" in q_norm or "kaygan" in q_norm:
                base = "yağmurda ıslak kaygan zeminde güvenli sürüş takip mesafesi aquaplaning"
            elif "sis" in q_norm or "pus" in q_norm:
                base = "sisli havada güvenli sürüş görüş mesafesi"
            elif "dur-kalk" in q_norm or "yoğun" in q_norm:
                base = "yoğun trafikte dur kalk trafikte güvenli sürüş dikkat takip mesafesi"
            elif "yorgun" in q_norm or "uzun yol" in q_norm or "mola" in q_norm:
                base = "uzun yolda yorgunluk dikkat mola güvenli sürüş"
        return base

    def _deterministic_final_benchmark(self, question: str,
                                       traffic_res: Optional[Dict[str, Any]],
                                       route_res: Optional[Dict[str, Any]],
                                       rag_res: Optional[Dict[str, Any]],
                                       math_pack: Optional[Dict[str, Any]]) -> str:
        qn = tr_normalize(question)
        out: List[str] = []

        # 1) Trafik özeti
        if traffic_res and traffic_res.get("ok"):
            t = traffic_res.get("traffic") or {}
            cong = t.get("congestion_pct")
            st = t.get("status")
            cur = t.get("currentSpeed")
            free = t.get("freeFlowSpeed")
            out.append(f"Anlık trafik: yoğunluk %{cong} ({st}).")
            if cur is not None and free is not None:
                out.append(f"Hız: {cur} km/s, serbest akış: {free} km/s.")
            elif traffic_res.get("provider") == "MOCK":
                out.append("Not: TOMTOM_API_KEY olmadığı için MOCK trafik verisi kullanıldı.")
            if cong is None:
                out.append("Not: congestion_pct verisi None/null geldi; bunu raporluyorum ve güvenli varsayımla ilerliyorum.")

        # 2) Rota özeti
        if route_res:
            if route_res.get("ok"):
                out.append(f"\nRota: {route_res.get('start')} → {route_res.get('end')}")
                routes_sorted = self._routes_sorted(route_res)
                if routes_sorted:
                    out.append("Rota(lar) süre/mesafe/gecikme:")
                    for r in routes_sorted:
                        out.append(f"- Alternatif {r.get('alternative')}: {r.get('travel_time_min')} dk, {r.get('length_km')} km, gecikme {r.get('traffic_delay_min')} dk")
                else:
                    out.append("Not: route listesi boş geldi; bunu raporluyorum.")
            else:
                out.append(f"\nRota bilgisi alınamadı: {route_res.get('error')}")

        # 3) RAG önerileri
        bullets = self._pick_top_safety_bullets(rag_res or {}, k=5) if rag_res else []
        want_5 = "5 öneri" in qn or "5 oner" in qn
        want_3 = "3 öneri" in qn or "3 oner" in qn
        want_2 = "2 öneri" in qn or "2 oner" in qn
        k = 4
        if want_5:
            k = 5
        elif want_3:
            k = 3
        elif want_2:
            k = 2

        if rag_res:
            out.append(f"\nRAG ({rag_res.get('doc_type')}):")
            if bullets:
                for i, b in enumerate(bullets[:k], 1):
                    out.append(f"{i}. {b}")
            else:
                out.append("Bu sorguya uygun net madde bulunamadı (snippet yapısı madde içermiyor olabilir).")

        # 4) Math sonuçları
        if math_pack and isinstance(math_pack, dict):
            comp = math_pack.get("computed", {})
            if comp:
                out.append("\nHesaplamalar (math):")
                if "follow_distance_m" in comp:
                    out.append(f"- Takip mesafesi (hız/2): {comp['follow_distance_m']} m")
                if "follow_distance_buffered_m" in comp:
                    out.append(f"- Tamponlu takip mesafesi: {comp['follow_distance_buffered_m']} m")
                if "follow_distance_base_m" in comp and "follow_distance_plus10_m" in comp:
                    out.append(f"- İki olasılık: base={comp['follow_distance_base_m']} m, base+10={comp['follow_distance_plus10_m']} m")
                if "delay_pct" in comp:
                    out.append(f"- Gecikme yüzdesi: {comp['delay_pct']} %")
                if "delay_per_km" in comp:
                    out.append(f"- Gecikme/mesafe (dk/km): {comp['delay_per_km']}")
                if "score" in comp:
                    out.append(f"- Skor: {comp['score']}")
                if "stress" in comp:
                    out.append(f"- Stres puanı: {comp['stress']}")
                if "risk" in comp:
                    out.append(f"- Risk skoru: {comp['risk']}")
                if "route_scores" in comp and comp["route_scores"]:
                    out.append("- Rota puanları:")
                    for rs in comp["route_scores"]:
                        out.append(f"  * Alt {rs['alternative']}: {rs['score']}")
                if "risk_flag" in comp:
                    flag = comp["risk_flag"]
                    out.append(f"- Risk etiketi koşulu: {flag} (True=Yüksek, False=Orta/Düşük)")

        # 5) Kısa gerekçe
        out.append("\nKısa gerekçe:")
        out.append("Araçlardan gelen anlık trafik/rota verisi sürüşün stres ve hata toleransını etkiler; bu yüzden önerileri (takip mesafesi, dikkat, hız kontrolü, öngörü) mevcut yoğunluk/gecikme ile ilişkilendirerek uygulamak gerekir. Kural/işaret bağlamı istenen senaryolarda ise 'gecikme var' diye kurallar gevşetilmez; aksine güvenli tarafta kalmak gerekir.")

        return "\n".join(out)

    # ============================================================
    # MAIN answer()
    # ============================================================
    def answer(self, question: str, max_turns: int = 5, return_trace: bool = True,
               required_tools: Optional[List[str]] = None,
               final_mode: str = "deterministic") -> Dict[str, Any]:
        """
        final_mode:
          - "deterministic": senin mevcut stabil benchmark finalin
          - "gemini": aynı tool observation'ları ile Gemini final yazdırır
        """
        required_tools = required_tools or []
        required_set = set(required_tools)

        trace: List[TraceStep] = []
        tools_used: List[str] = []
        contexts_used: List[Dict[str, Any]] = []
        evidence_texts: List[str] = []

        q_norm = tr_normalize(question)

        # needs flags
        if required_tools:
            needs_rag = ("rag_search" in required_set)
            needs_math = ("math" in required_set)
            needs_live_traffic = ("live_traffic_api" in required_set)
            needs_route_eta = ("route_eta_api" in required_set)
        else:
            needs_rag = any(k in q_norm for k in ["yağmur", "sis", "yönetmelik", "kanun", "trafik işaret", "levha"])
            needs_math = any(k in q_norm for k in ["hız", "yarısı", "metre", "%", "puan", "skor", "oran", "risk", "stres"])
            needs_live_traffic = any(k in q_norm for k in ["şu an", "anlık", "yoğunluk", "trafik durumu"])
            needs_route_eta = any(k in q_norm for k in ["rota", "eta", "kaç dakika", "sürer", "güzergah", "güzergâh"])

        # ============================================================
        # ✅ BENCHMARK MODE (required_tools provided):
        # Force-run tools + final_mode
        # ============================================================
        if required_tools:
            route_res = None
            traffic_res = None
            rag_res = None
            math_pack = None

            turn_idx = 0

            # 1) Forced: route
            if needs_route_eta:
                start, end = self._extract_route_points(question)
                args = {"start": start, "end": end}
                tools_used.append("route_eta_api")
                route_res = self.tools["route_eta_api"](args)
                obs = safe_json(route_res)
                evidence_texts.append(obs)
                trace.append(TraceStep(turn_idx, "Benchmark forced: route_eta_api",
                                       f"route_eta_api {json.dumps(args, ensure_ascii=False)}",
                                       obs[:800] + ("..." if len(obs) > 800 else "")))
                turn_idx += 1

            # 2) Forced: live traffic
            if needs_live_traffic:
                area = "Esenyurt, İstanbul"
                m_area = re.search(r"(\b[A-Za-zÇĞİÖŞÜçğıöşü]+)\s+bölgesinde", question)
                if m_area:
                    area = self._clean_place(m_area.group(1))
                args = {"area": area}
                tools_used.append("live_traffic_api")
                traffic_res = self.tools["live_traffic_api"](args)
                obs = safe_json(traffic_res)
                evidence_texts.append(obs)
                trace.append(TraceStep(turn_idx, "Benchmark forced: live_traffic_api",
                                       f"live_traffic_api {json.dumps(args, ensure_ascii=False)}",
                                       obs[:800] + ("..." if len(obs) > 800 else "")))
                turn_idx += 1

            # 3) Forced: rag
            if needs_rag:
                doc_type = "SAFETY"
                if "SIGNS" in question.upper() or "işaret" in q_norm or "levha" in q_norm:
                    doc_type = "SIGNS"
                elif "REGULATION" in question.upper() or "yönetmelik" in q_norm or "kanun" in q_norm:
                    doc_type = "REGULATION"

                q = self._benchmark_rag_query(question, q_norm, doc_type)
                args = {"query": q, "doc_type": doc_type, "top_k": 6}
                tools_used.append("rag_search")
                rag_res = self.tools["rag_search"](args)
                obs = safe_json(rag_res)
                evidence_texts.append(obs)

                if rag_res.get("ok"):
                    for r in (rag_res.get("results") or [])[:3]:
                        evidence_texts.append((r.get("text") or "")[:2000])

                    contexts_used.append({
                        "doc_type": rag_res.get("doc_type"),
                        "best_score": rag_res.get("best_score"),
                        "top_snippets": [{
                            "score": r.get("score"),
                            "source": r.get("meta", {}).get("source"),
                            "chunk_id": r.get("meta", {}).get("chunk_id"),
                            "text_preview": (r.get("text", "")[:260] + "...")
                        } for r in (rag_res.get("results") or [])[:3]]
                    })

                trace.append(TraceStep(turn_idx, "Benchmark forced: rag_search",
                                       f"rag_search {json.dumps(args, ensure_ascii=False)}",
                                       obs[:800] + ("..." if len(obs) > 800 else "")))
                turn_idx += 1

            # 4) Forced: math (soruya göre gerekli tüm hesapları yap)
            if needs_math:
                math_pack = self._benchmark_math_pack(question, traffic_res or {}, route_res or {},
                                                      tools_used, trace, evidence_texts, start_turn=turn_idx)
                turn_idx = int(math_pack.get("turn_end", turn_idx))

            # 5) Final üret (A/B)
            if final_mode == "gemini":
                final_text = self._gemini_final_benchmark(question, traffic_res, route_res, rag_res, math_pack, contexts_used)
                if not final_text:
                    final_text = "(Gemini boş cevap döndürdü.)"
            else:
                final_text = self._deterministic_final_benchmark(question, traffic_res, route_res, rag_res, math_pack)

            # Evidence guard
            bad, unsupported = self._final_has_unsupported_numbers(final_text, evidence_texts)
            if bad:
                for n in unsupported:
                    final_text = re.sub(rf"\b{re.escape(n)}\b", "", final_text)
                final_text = re.sub(r"\s{2,}", " ", final_text).strip()

            return {
                "ok": True,
                "answer": final_text,
                "trace": [t.__dict__ for t in trace] if return_trace else [],
                "tools_used": tools_used,
                "contexts_used": contexts_used
            }

        # ============================================================
        # Non-benchmark (demo vs.) - real ReAct loop (FIXED)
        # ============================================================
        prompt = f"SORU: {question}\n"
        last_action = None

        for turn in range(1, max_turns + 1):
            llm_out = self._llm(prompt)

            m_final = re.search(
                r"(FINAL\s*ANSWER|FINAL|Answer|Cevap)\s*:\s*(.*)",
                llm_out,
                re.IGNORECASE | re.DOTALL
            )
            if m_final:
                final_text = m_final.group(2).strip()
                bad, unsupported = self._final_has_unsupported_numbers(final_text, evidence_texts)
                if bad:
                    final_text = re.sub(r"\d+(?:[.,]\d+)?", "", final_text)
                    final_text = re.sub(r"\s{2,}", " ", final_text).strip()

                return {
                    "ok": True,
                    "answer": final_text,
                    "trace": [t.__dict__ for t in trace] if return_trace else [],
                    "tools_used": tools_used,
                    "contexts_used": contexts_used
                }

            m = re.search(r"(ACTION|Action)\s*:\s*([a-zA-Z_]+)\s*(\{.*\})", llm_out, re.DOTALL)
            if not m:
                trace.append(TraceStep(turn, "LLM çıktı ama ACTION/FİNAL yok", "NONE", llm_out[:800]))
                return {"ok": False, "answer": "ACTION çözümlenemedi (LLM formatı bozuk).", "trace": [t.__dict__ for t in trace], "tools_used": tools_used, "contexts_used": contexts_used}

            tool = m.group(2).strip()
            js = m.group(3).strip()

            try:
                args = json.loads(js)
            except Exception:
                args = {}

            if tool not in self.tools:
                trace.append(TraceStep(turn, "Tool bulunamadı", "NONE", f"Tool yok: {tool}"))
                return {"ok": False, "answer": f"Tool bulunamadı: {tool}", "trace": [t.__dict__ for t in trace], "tools_used": tools_used, "contexts_used": contexts_used}

            tools_used.append(tool)
            result = self.tools[tool](args)
            obs_text = safe_json(result)
            evidence_texts.append(obs_text)

            trace.append(TraceStep(turn, "ReAct tool çağrısı", f"{tool} {json.dumps(args, ensure_ascii=False)}", obs_text[:800] + ("..." if len(obs_text) > 800 else "")))

            current_action = (tool, json.dumps(args, ensure_ascii=False, sort_keys=True))
            if current_action == last_action:
                prompt += "\nUyarı: Aynı aracı tekrar çağırdın. FINAL ANSWER üret.\n"
            last_action = current_action

            prompt = (
                f"SORU: {question}\n\n"
                f"OBSERVATION(JSON): {obs_text}\n\n"
                "Şimdi sadece kurallara uyarak devam et.\n"
            )

        return {"ok": False, "answer": "Maksimum adım sayısına ulaşıldı (max_turns).", "trace": [t.__dict__ for t in trace], "tools_used": tools_used, "contexts_used": contexts_used}


# ============================================================
# 6) build_system
# ============================================================

def build_system() -> SmartTrafficReActAgent:
    rag = MultiRAG()

    REG_YON_PDF = "/content/REG_trafik_yonetmeligi.pdf"
    REG_2918_PDF = "/content/REG_2918.pdf"
    SIGNS_PDF = "/content/REG_TrafikIsaretleriElKitabi.pdf"
    SAFETY_PDF = "/content/SAFETY_Guvenli_Surus_Teknikleri.pdf"

    for p in [REG_YON_PDF, REG_2918_PDF, SIGNS_PDF, SAFETY_PDF]:
        if not os.path.exists(p):
            raise FileNotFoundError(f"PDF bulunamadı: {p} | /content altını kontrol et.")

    rag.ingest_pdf(SAFETY_PDF, doc_type="SAFETY", chunk_size=950, overlap=160)
    rag.ingest_pdf(REG_YON_PDF, doc_type="REGULATION", chunk_size=950, overlap=160)
    rag.ingest_pdf(REG_2918_PDF, doc_type="REGULATION", chunk_size=950, overlap=160)
    rag.ingest_pdf(SIGNS_PDF, doc_type="SIGNS", chunk_size=950, overlap=160)

    return SmartTrafficReActAgent(rag=rag)


# ============================================================
# 7) Benchmark / Senaryo Testleri
# ============================================================

BENCHMARK_CASES = [
  {
    "id": "B01",
    "question": "Esenyurt’ta anlık trafik yoğunluğunu çek. Yoğunluk %25 üstüyse SAFETY dokümanından dur-kalk trafikte risk azaltmaya yönelik 4 öneri çıkar. Sonra 70 km/s için 'hızın yarısı kadar metre' kuralıyla takip mesafesini hesapla ve yoğunlukla ilişkilendirerek kısa gerekçe yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B02",
    "question": "Şu an Beşiktaş’ta trafik yoğunluğunu kontrol et. Yoğunluk %40 ve üstüyse SAFETY’den 4 madde güvenli sürüş önlemi getir. 90 km/s için takip mesafesini hesapla. Sonuçta 'neden özellikle bu önlemler' diye kısa gerekçe yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B03",
    "question": "Kadıköy anlık trafik durumunu çek. Eğer durum 'Yoğun' veya 'Çok Yoğun' ise SAFETY’den dikkat/öngörü/hız temalı 4 öneri çıkar. 50 km/s için takip mesafesini hesapla. Trafik akıcıysa şartın tetiklenmediğini açıkla ve yine de güvenli sürüş açısından kısa yorum yap.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B04",
    "question": "Ataşehir’de anlık yoğunluğu çek. Yoğunluk yüzdesine göre (0–14, 15–34, 35+) sürüş stratejisi yaz: her seviye için SAFETY’den 2 öneri (toplam 6). 80 km/s takip mesafesini hesapla ve seviyelere göre 'arttır/koru' kararını gerekçelendir.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B05",
    "question": "Bakırköy’de anlık yoğunluğu çek. Yoğunluk %35+ ise SAFETY’den 4 öneri çıkar ve özellikle ani fren/dur-kalk riskine dair bağlam getir. 60 km/s takip mesafesini hesapla ve her önerinin hangi riski azalttığını 1-2 cümleyle eşleştir.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B06",
    "question": "Şişli’de anlık trafik durumunu ve yoğunluk yüzdesini çek. SAFETY’den ıslak/kaygan zemin ile ilgili 3 öneri ve genel yoğun trafikte dikkat ile ilgili 1 öneri getir. 90 km/s takip mesafesini hesapla ve 4 öneriyi tek bir kısa sürüş planı halinde birleştir.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B07",
    "question": "Taksim’den Kadıköy’e trafik etkili rota ve alternatifleri çek. En hızlı rotanın gecikmesi 8 dk üstüyse SAFETY’den 4 öneri getir. Ayrıca gecikme/mesafe oranını (gecikme dk / km) hesaplayıp gecikmenin sürücü riski/yorulma etkisini 2-3 cümleyle gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B08",
    "question": "Beşiktaş’tan Üsküdar’a rota+ETA al (alternatif dahil). En hızlı rota gecikmesi 5 dk üstüyse SAFETY’den sabır/takip mesafesi/şerit disiplini temalı 4 öneri getir. En hızlı ve en kısa mesafeli rotayı karşılaştırıp hangisini seçtiğini net kriterle gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B09",
    "question": "Esenyurt’tan Taksim’e rota ve alternatifleri çek. Eğer alternatif rotaların süre farkı 10 dk’dan büyükse SAFETY’den dur-kalk ve dikkat dağınıklığı riskine dair 4 öneri getir. Süre farkını hesapla ve 'daha stabil rota' seçimini gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B10",
    "question": "Kadıköy’den Ataşehir’e rota+ETA al. En hızlı rotanın gecikmesi toplam sürenin %15’inden fazlaysa SAFETY’den 4 öneri getir. Gecikme yüzdesini hesapla ve kararını kısa gerekçeyle yaz.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B11",
    "question": "Bakırköy’den Şişli’ye rota sonuçlarını çek. En hızlı rotada gecikme 0 olsa bile SAFETY’den şehir içi dikkat için 2 öneri + takip mesafesi için 2 öneri getir. 70 km/s takip mesafesini hesapla ve rota ile ilişkilendirerek kısa plan yaz.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B12",
    "question": "Üsküdar’dan Taksim’e rota ve alternatifleri çek. Eğer en hızlı rotanın gecikmesi 8 dk üstüyse SAFETY’den 4 öneri getir. Ayrıca basit bir skor üret: skor = gecikme * 2. Skoru hesapla ve neden bu skorun sürücü davranışını etkilemesi gerektiğini gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B13",
    "question": "(1) Esenyurt’ta anlık yoğunluğu çek. (2) Esenyurt’tan Taksim’e rota+ETA çek. Eğer yoğunluk %35+ VE rota gecikmesi 8+ ise SAFETY’den 5 öneri getir. İki risk birden yüksekken neden daha kritik olduğunu 3 cümle ile açıkla.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B14",
    "question": "Kadıköy’de anlık yoğunluğu al, sonra Kadıköy’den Beşiktaş’a rota+ETA çek. Yoğunluk %25+ ise 80 km/s takip mesafesini hesapla. Rota gecikmesi de 8+ ise SAFETY’den 4 öneri getir ve hepsini tek bir sürüş planı halinde gerekçelendir.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B15",
    "question": "Şişli’de anlık yoğunluğu al, Şişli’den Bakırköy’e rota+ETA çek. Basit stres puanı tanımla: stres = yoğunluk + (gecikme*5). Değerleri çekip stresi hesapla. SAFETY’den 4 öneri getir ve stres puanını gerekçelendirmede kullan.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B16",
    "question": "Beşiktaş yoğunluğunu ve Beşiktaş’tan Üsküdar’a rota gecikmesini al. Yoğunluk düşük ama gecikme yüksekse (veya tersi) hangi durumda daha temkinli olacağını SAFETY bağlamıyla tartış. 90 km/s takip mesafesini hesaplayıp kararına ekle.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B17",
    "question": "Ataşehir yoğunluğunu ve Ataşehir’den Kadıköy’e rota+ETA çek. Yoğunluk %15–34 arasıysa SAFETY’den 3 öneri, %35+ ise 5 öneri getir. 60 km/s takip mesafesini hesapla ve hangi eşikte hangi önerilerin devreye girdiğini kısa listede göster.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B18",
    "question": "Bakırköy yoğunluğunu ve Bakırköy’den Taksim’e rota+ETA çek. En hızlı rota ile en az gecikmeli rota farklıysa seçim puanı uygula: puan = süre + 2*gecikme. Her rota için puanı hesapla, en düşük puanlı rotayı seç ve SAFETY’den 4 öneriyle gerekçelendir.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B19",
    "question": "Taksim’den Kadıköy’e rota+ETA çek. SIGNS dokümanından hız limitleri/uyarı levhalarıyla ilgili bağlam getir. Rota gecikmesi varsa sürücüde 'hız baskısı' riskini tartış ve 90 km/s takip mesafesini hesaplayıp neden hız yapılmaması gerektiğini bağlamla açıkla.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B20",
    "question": "Üsküdar’dan Beşiktaş’a rota+ETA çek. REGULATION dokümanından takip mesafesi/güvenli sürüş ile ilgili bağlam getir. 70 km/s takip mesafesini hesapla. 'Rota gecikmesi sürücüyü zorlar ama kural değişmez' temasında kısa karar yaz.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B21",
    "question": "Esenyurt’ta anlık yoğunluğu çek. SIGNS dokümanından dikkat/uyarı levhalarının amacıyla ilgili bağlam getir. 50 km/s takip mesafesini hesapla. Uyarı levhası mantığı ile yoğun trafikte risk yönetimini birleştirerek kısa açıklama yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B22",
    "question": "Kadıköy’de anlık yoğunluğu çek. REGULATION dokümanından şehir içi hız/kurallar hakkında bağlam getir. 90 km/s takip mesafesini hesapla ve şehir içinde hız baskısının neden riskli olduğunu kanıta dayalı yorumla (dokümanda net sayı yoksa bunu belirt).",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B23",
    "question": "Esenyurt’ta anlık yoğunluğu çek. SAFETY dokümanından yağmur/ıslak zeminle ilgili bağlam getir. Eğer yoğunluk %35+ ise hedef hızı %20 düşür: örneğin 80→64. Bu hesabı yap ve nedenini bağlamla açıkla.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B24",
    "question": "Taksim’den Kadıköy’e rota gecikmesini al. Eğer gecikme 8+ ise 'mola önerisi' üret: mola_dk = gecikme/2. Hesapla. SAFETY’den yorgunluk/dikkat bağlamı getir ve öneriyi gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B25",
    "question": "Şişli’de anlık yoğunluğu al. Durum 'Yoğun/Çok Yoğun' ise güvenli takip mesafesine +%30 tampon ekle: tamponlu = (hız/2)*1.3 (hız=60). Hesapla. SAFETY’den takip mesafesi ve yoğun trafik bağlamı getirip yorumla.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B26",
    "question": "Beşiktaş’tan Üsküdar’a rota+ETA çek. Gecikme varsa risk skoru hesapla: risk = gecikme*10 + (süre/10). Hesapla. SAFETY’den 4 öneri getir ve risk skoruna göre ciddiyet seviyesini yorumla.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B27",
    "question": "Ataşehir’de anlık yoğunluğu al. Eğer yoğunluk %15–34 ise 70 km/s için takip mesafesini (hız/2) hesapla. Eğer %35+ ise (hız/2)+10 metre yap. Math ile iki ihtimali de hesapla, sonra gerçek yoğunluğa göre doğru olanı seçip SAFETY bağlamıyla gerekçelendir.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B28",
    "question": "Bakırköy’den Taksim’e rota+alternatifleri çek. Seçim kuralı: puan = süre + 3*gecikme (en düşük puan seçilir). Her rota için puanı hesapla. SAFETY’den stres/acele ve yoğun trafikte dikkat bağlamı getirip seçimini gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B29",
    "question": "Taksim’den Kadıköy’e rota+ETA çek. Eğer API rota döndürmezse: (1) bunu açıkça belirt, (2) SAFETY’den şehir içi yoğun trafikte sürüş bağlamı getir, (3) 70 km/s takip mesafesini hesapla ve 'alternatif veri yokken güvenli varsayım' gerekçesi yaz.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B30",
    "question": "Esenyurt’ta anlık yoğunluğu çek. Eğer congestion_pct None/null gelirse bunu raporla. Sonra SAFETY’den 4 öneri getir ve 90 km/s takip mesafesini hesapla. 'Veri eksikken nasıl karar verdin?' sorusunu 2 cümleyle açıkla.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B31",
    "question": "Beşiktaş’tan Üsküdar’a rota+ETA çek. Eğer route listesi boş gelirse bunu belirt. REGULATION’dan genel trafik kuralları bağlamı getir. 60 km/s takip mesafesini hesapla ve 'rota yoksa bile sürüş prensibi' diye kısa karar yaz.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B32",
    "question": "Kadıköy’de anlık yoğunluğu çek. Eğer status 'Akıcı' ama congestion_pct yüksek gibi tutarsız veri gelirse tutarsızlığı belirt. SAFETY’den 4 öneri getir ve 50 km/s takip mesafesini hesapla. 'Tutarsız veride güvenli tarafta kalma' prensibiyle karar yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B33",
    "question": "İstanbul (Taksim) → Kocaeli (İzmit Merkez) rota ve alternatifleri çek. Alternatifler arasında gecikme farkı 15 dk üstüyse SAFETY’den uzun yolda dikkat/yorgunluk bağlamı getir. Gecikme farkını hesapla, en düşük gecikmeli rotayı seç ve gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B34",
    "question": "Ankara (Kızılay) → Eskişehir (Merkez) rota+alternatifleri çek. Seçim kuralı: puan = süre + 2*gecikme. Her rota için puanı hesapla. SAFETY’den uzun yol sürüş önerileri getir ve seçimini kanıta dayalı gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B35",
    "question": "Adana (Merkez) → Mersin (Merkez) rota+alternatif çek. En hızlı rota gecikmesi 10 dk üstüyse SAFETY’den 4 öneri getir. Gecikmenin toplam süreye oranını hesapla ve neden risk/stres arttığını kısa gerekçeyle açıkla.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B36",
    "question": "İzmir (Konak) → Manisa (Merkez) rota+alternatif çek. Alternatif rotalarda mesafe farkı varsa km farkını hesapla. SAFETY’den dikkat/öngörü bağlamı getir ve yakıt/yorulma açısından hangi rotayı seçtiğini gerekçelendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B37",
    "question": "Esenyurt’ta anlık yoğunluğu çek. SIGNS dokümanından uyarı/tehlike levhalarının amacıyla ilgili bağlam getir. 70 km/s takip mesafesini hesapla. Yoğunluk yüksekse uyarı levhası mantığı ile risk planını birleştirerek kısa açıklama yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B38",
    "question": "Şişli’de anlık yoğunluğu çek. REGULATION dokümanından şehir içi sürüş kuralları hakkında bağlam getir. 60 km/s takip mesafesini hesapla. Kural + anlık durum + hesap ile kısa ama gerekçeli sürüş kararı yaz.",
    "expected_tools": ["live_traffic_api", "rag_search", "math"]
  },
  {
    "id": "B39",
    "question": "Taksim’den Kadıköy’e rota+ETA çek. SIGNS dokümanından hız/uyarı ile ilgili bağlam getir. Eğer gecikme 8+ ise sürücünün 'acele etme' riskini 3 maddeyle açıkla, 90 km/s takip mesafesini hesapla ve bağlamla ilişkilendir.",
    "expected_tools": ["route_eta_api", "rag_search", "math"]
  },
  {
    "id": "B40",
    "question": "Kadıköy’de yoğunluğu çek ve Kadıköy’den Beşiktaş’a rota+ETA çek. Risk etiketi üret: (yoğunluk>=35 veya gecikme>8) -> 'Yüksek', aksi -> 'Orta/Düşük'. Bu mantığı math ile işlet (1/0 gibi). SAFETY’den 4 öneri getir ve etiketi kanıta dayalı gerekçelendir.",
    "expected_tools": ["live_traffic_api", "route_eta_api", "rag_search", "math"]
  }
]

def run_benchmark(agent: SmartTrafficReActAgent, max_turns: int = 5, final_mode: str = "deterministic"):
    results = []
    for case in BENCHMARK_CASES:
        out = agent.answer(
            case["question"],
            max_turns=max_turns,
            return_trace=True,
            required_tools=case.get("expected_tools", []),
            final_mode=final_mode
        )
        results.append({
            "id": case["id"],
            "ok": out["ok"],
            "question": case["question"],
            "answer": out.get("answer", ""),
            "answer_preview": (out["answer"][:220] + "...") if out.get("answer") else "",
            "tools_used": out.get("tools_used", []),
            "expected_tools": case.get("expected_tools", []),
            "trace": out.get("trace", []),
            "contexts_used": out.get("contexts_used", []),
            "final_mode": final_mode
        })
    return results

def benchmark_summary(results):
    total = len(results)
    ok_count = sum(1 for r in results if r.get("ok"))
    tool_match_count = sum(
        1 for r in results
        if set(r.get("expected_tools", [])).issubset(set(r.get("tools_used", [])))
    )
    avg_turns = round(sum(len(r.get("trace", [])) for r in results) / total, 2) if total else 0
    return {
        "Toplam Senaryo": total,
        "Cevap Başarı (ok) %": round(100 * ok_count / total, 2) if total else 0,
        "Tool Doğruluk (expected ⊆ used) %": round(100 * tool_match_count / total, 2) if total else 0,
        "Ortalama Adım Sayısı": avg_turns
    }

def compare_benchmarks(det: List[Dict[str, Any]], gem: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    gem_map = {x["id"]: x for x in gem}
    out = []
    for d in det:
        g = gem_map.get(d["id"], {})
        out.append({
            "id": d["id"],
            "question": d["question"],
            "expected_tools": d.get("expected_tools", []),
            "tools_used_det": d.get("tools_used", []),
            "tools_used_gem": g.get("tools_used", []),
            "det_preview": d.get("answer_preview", ""),
            "gem_preview": g.get("answer_preview", ""),
            "det_answer": d.get("answer", ""),
            "gem_answer": g.get("answer", ""),
        })
    return out


# ============================================================
# 8) Run (A/B)
# ============================================================

if __name__ == "__main__":
    # Gereken env:
    #   export TOMTOM_API_KEY="..."
    #   export GEMINI_API_KEY="..."
    # Opsiyonel:
    #   export GEMINI_MODEL="models/gemini-2.5-flash"
    #   export GROQ_API_KEY="..." (demo loop için)

    agent = build_system()

    # A) Senin final (deterministic)
    bench_det = run_benchmark(agent, max_turns=5, final_mode="deterministic")
    print("\n================= BENCHMARK SUMMARY (DETERMINISTIC) =================")
    summary_det = benchmark_summary(bench_det)
    for k, v in summary_det.items():
        print(f"{k}: {v}")

    # B) Gemini final
    bench_gem = run_benchmark(agent, max_turns=5, final_mode="gemini")
    print("\n================= BENCHMARK SUMMARY (GEMINI) =================")
    summary_gem = benchmark_summary(bench_gem)
    for k, v in summary_gem.items():
        print(f"{k}: {v}")

    # Yan yana karşılaştırma (JSON)
    comp = compare_benchmarks(bench_det, bench_gem)

    print("\n================= BENCHMARK COMPARE (DET vs GEMINI) =================")
    for row in comp:
        print(f"\n--- Case {row['id']} ---")
        print("Expected:", row["expected_tools"])
        print("\n[DET] Preview:", row["det_preview"])
        print("\n[GEM] Preview:", row["gem_preview"])

    # İstersen tam çıktıları kaydet:
    # with open("bench_det.json", "w", encoding="utf-8") as f:
    #     json.dump(bench_det, f, ensure_ascii=False, indent=2)
    # with open("bench_gem.json", "w", encoding="utf-8") as f:
    #     json.dump(bench_gem, f, ensure_ascii=False, indent=2)
    # with open("bench_compare.json", "w", encoding="utf-8") as f:
    #     json.dump(comp, f, ensure_ascii=False, indent=2)



================= BENCHMARK SUMMARY (DETERMINISTIC) =================
Toplam Senaryo: 40
Cevap Başarı (ok) %: 100.0
Tool Doğruluk (expected ⊆ used) %: 70.0
Ortalama Adım Sayısı: 3.02

================= BENCHMARK SUMMARY (GEMINI) =================
Toplam Senaryo: 40
Cevap Başarı (ok) %: 100.0
Tool Doğruluk (expected ⊆ used) %: 70.0
Ortalama Adım Sayısı: 3.02

================= BENCHMARK COMPARE (DET vs GEMINI) =================

--- Case B01 ---
Expected: ['live_traffic_api', 'rag_search', 'math']

[DET] Preview: Anlık trafik: yoğunluk %18 (Orta).
Hız: 32 km/s, serbest akış: 39 km/s.

RAG (SAFETY):
1. Önce güvenliği düşünün.
2. Emniyet kemerinizi takın
3. Aynalarınız doğru açıları göstersin
4. Sinyalleri Doğru Kullanın

Hesaplama...

[GEM] Preview: Gemini aktif değil (GEMINI_API_KEY yok veya google-generativeai yüklü değil)....

--- Case B02 ---
Expected: ['live_traffic_api', 'rag_search', 'math']

[DET] Preview: Anlık trafik: yoğunluk %18 (Orta).
Hız: 32 km/s, serbest akış: 39 km/s.



In [8]:
!pip -q install groq pandas

from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY","").strip())
import time, json
import pandas as pd

def judge_with_llm(question, agent_answer, expected_tools, used_tools, contexts_used):
    """
    LLM-as-a-Judge: Senin proje bağlamına göre kalite puanı (0-100).
    """
    ctx_hint = ""
    if contexts_used:
        # sadece kaynak isimlerini özet geç
        sources = []
        for c in contexts_used:
            for sn in c.get("top_snippets", []):
                if sn.get("source"):
                    sources.append(sn["source"])
        sources = list(dict.fromkeys(sources))[:5]
        if sources:
            ctx_hint = f"\nContexts (sources): {', '.join(sources)}"

    judge_prompt = f"""
You are an impartial Quality Assurance Auditor for a Smart Transportation ReAct Agent.

Evaluation Criteria:
1) Tool Correctness: Did the agent use the right tools for this question?
   - Expected tools: {expected_tools}
   - Used tools: {used_tools}
2) Answer Quality: Does the final answer address the question clearly and correctly?
3) Source Fidelity: If RAG was used, is the answer consistent with retrieved context (no hallucination)?

User Question: "{question}"
Agent Final Answer: "{agent_answer}"
{ctx_hint}

TASK: Return a single integer score between 0 and 100.
Return ONLY the integer number, no text.
""".strip()

    try:
        resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role":"user","content":judge_prompt}],
            temperature=0
        )
        score_text = resp.choices[0].message.content.strip()
        digits = ''.join(ch for ch in score_text if ch.isdigit())
        return int(digits) if digits else 0
    except Exception as e:
        print("Judge error:", e)
        return 0


def run_llm_judge_benchmark(agent, cases, max_turns=5, save_csv_path="/content/final_performance_report.csv"):
    final_results = []
    print(f"✅ STARTING LLM-JUDGE BENCHMARK ({len(cases)} cases)\n" + "="*70)

    for case in cases:
        q = case["question"]
        exp_tools = case.get("expected_tools", [])

        print(f"🔍 {case['id']}: {q}")
        start = time.time()

        out = agent.answer(q, max_turns=max_turns, return_trace=True)
        latency = round(time.time() - start, 2)

        ans = out.get("answer","")
        used_tools = out.get("tools_used", [])
        contexts_used = out.get("contexts_used", [])

        # tool doğruluk
        tool_ok = set(exp_tools).issubset(set(used_tools))

        # judge score
        score = judge_with_llm(q, ans, exp_tools, used_tools, contexts_used)

        final_results.append({
            "ID": case["id"],
            "Question": q,
            "OK": out.get("ok", False),
            "Expected_Tools": ", ".join(exp_tools),
            "Used_Tools": ", ".join(used_tools),
            "Tool_Correct": tool_ok,
            "Judge_Score": score,
            "Latency_Sec": latency,
            "Steps": len(out.get("trace", [])),
        })

        print(f"   ✅ Tool_Correct={tool_ok} | Judge={score} | Latency={latency}s | Steps={len(out.get('trace', []))}")

    df = pd.DataFrame(final_results)

    # summary
    avg_score = round(df["Judge_Score"].mean(), 2) if len(df) else 0
    avg_latency = round(df["Latency_Sec"].mean(), 2) if len(df) else 0
    tool_acc = round(100 * df["Tool_Correct"].mean(), 2) if len(df) else 0
    ok_rate = round(100 * df["OK"].mean(), 2) if len(df) else 0

    summary_row = {
        "ID": "SUMMARY",
        "Question": "AVERAGES",
        "OK": f"{ok_rate}%",
        "Expected_Tools": "-",
        "Used_Tools": "-",
        "Tool_Correct": f"{tool_acc}%",
        "Judge_Score": avg_score,
        "Latency_Sec": avg_latency,
        "Steps": round(df["Steps"].mean(), 2) if len(df) else 0
    }
    df_out = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

    df_out.to_csv(save_csv_path, index=False)

    print("\n" + "="*70)
    print("📌 LLM-JUDGE BENCHMARK SUMMARY")
    print(f"OK Rate: {ok_rate}%")
    print(f"Tool Accuracy: {tool_acc}%")
    print(f"Avg Judge Score: {avg_score}")
    print(f"Avg Latency: {avg_latency}s")
    print(f"Saved CSV: {save_csv_path}")
    print("="*70)

    display(df_out[["ID","OK","Tool_Correct","Judge_Score","Latency_Sec","Steps"]])
    return df_out


# ✅ Çalıştır (agent ana koddan geliyor olmalı)
agent = build_system()
df_report = run_llm_judge_benchmark(agent, BENCHMARK_CASES, max_turns=5, save_csv_path="/content/final_performance_report.csv")


✅ STARTING LLM-JUDGE BENCHMARK (40 cases)
🔍 B01: Esenyurt’ta anlık trafik yoğunluğunu çek. Yoğunluk %25 üstüyse SAFETY dokümanından dur-kalk trafikte risk azaltmaya yönelik 4 öneri çıkar. Sonra 70 km/s için 'hızın yarısı kadar metre' kuralıyla takip mesafesini hesapla ve yoğunlukla ilişkilendirerek kısa gerekçe yaz.
   ✅ Tool_Correct=False | Judge=0 | Latency=1.66s | Steps=0
🔍 B02: Şu an Beşiktaş’ta trafik yoğunluğunu kontrol et. Yoğunluk %40 ve üstüyse SAFETY’den 4 madde güvenli sürüş önlemi getir. 90 km/s için takip mesafesini hesapla. Sonuçta 'neden özellikle bu önlemler' diye kısa gerekçe yaz.
   ✅ Tool_Correct=False | Judge=0 | Latency=1.34s | Steps=0
🔍 B03: Kadıköy anlık trafik durumunu çek. Eğer durum 'Yoğun' veya 'Çok Yoğun' ise SAFETY’den dikkat/öngörü/hız temalı 4 öneri çıkar. 50 km/s için takip mesafesini hesapla. Trafik akıcıysa şartın tetiklenmediğini açıkla ve yine de güvenli sürüş açısından kısa yorum yap.
   ✅ Tool_Correct=False | Judge=85 | Latency=1.41s | Steps=0
🔍 B0

,ID,OK,Tool_Correct,Judge_Score,Latency_Sec,Steps
0,B01,True,False,0.00,1.66,0.0
1,B02,True,False,0.00,1.34,0.0
2,B03,True,False,85.00,1.41,0.0
3,B04,True,False,40.00,1.65,0.0
4,B05,True,False,0.00,1.59,0.0
5,B06,True,False,60.00,1.89,0.0
6,B07,True,False,0.00,1.80,0.0
7,B08,True,False,60.00,1.56,0.0
8,B09,True,False,85.00,1.69,0.0
9,B10,True,False,0.00,1.90,0.0
